<a href="https://colab.research.google.com/github/danielelbrecht/mirna/blob/master/mirna_model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import numpy as np

from keras.callbacks import LearningRateScheduler
from keras.models import Model
from keras.layers import Input, GRU, LSTM, TimeDistributed, Dropout, Dense, Permute, Flatten, Multiply, RepeatVector, Activation, Masking, Bidirectional
from keras import regularizers, optimizers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.wrappers import Wrapper
from keras.engine.topology import InputSpec
from keras import backend as K

In [4]:
# Load data sets

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

pos_file_obj = drive.CreateFile({'id': '1vl-qE0U5W6ll3JH41QqDajyx6oAwC3C0'})                       
pos_file_obj.GetContentFile('input.txt')

neg_file_obj = drive.CreateFile({'id': '1Rnh8RHUsmCGmiCZobu3g7ezeUJQq0CH-'})                       
neg_file_obj.GetContentFile('negatives.txt')

    100% |████████████████████████████████| 993kB 23.8MB/s 


In [0]:
pos_file_obj
pos_content = pos_file_obj.GetContentString()
neg_content = neg_file_obj.GetContentString()
pos_file = []
neg_file = []
temp = []

for x in pos_content:
  if x == '\n':
    pos_file.append(temp)
    temp = []
  else: 
    temp.append(x)
    
for x in neg_content:
  if x == '\n':
    neg_file.append(temp)
    temp = []
  else: 
    temp.append(x)
    
    
def read_line(line):

    array = []

    for entry in line:
        if entry == '0':
            array.append(np.int32(0))
        if entry == '1':
            array.append(np.int32(1))
        if len(array) == 16:
          break

    return np.asarray(array)
  
def process_data(pos_file, neg_file):

    data = []
    is_example = 0
    pos_examples = 0
    neg_examples = 0

    for line in pos_file: # Iterate over file

        if (line[0] == '0' or line[0] == '1') and is_example == 0:  # When new sequence is encountered, initialize new example
            example = []
            is_example = 1
            example.append(read_line(line))

        if (line[0] == '0' or line[0] == '1') and is_example == 1:  # During sequence
            example.append(read_line(line))

        if line[0] != '0' and line[0] != '1' and is_example == 1:  # When sequence terminates
            is_example = 0
            data.append(example)
            pos_examples = pos_examples + 1

    for line in neg_file: # Iterate over file

        if (line[0] == '0' or line[0] == '1') and is_example == 0:  # When new sequence is encountered, initialize new example
            example = []
            is_example = 1
            example.append(read_line(line))

        if (line[0] == '0' or line[0] == '1') and is_example == 1:  # During sequence
            example.append(read_line(line))

        if line[0] != '0' and line[0] != '1' and is_example == 1:  # When sequence terminates
            is_example = 0
            data.append(example)
            neg_examples = neg_examples + 1

    return np.asarray(data), pos_examples, neg_examples

In [0]:
# Process the data and generate training labels

full_data, num_pos, num_neg = process_data(pos_file, neg_file)

# Generate labels
pos_labels = np.ones(num_pos)
neg_labels = np.zeros(num_neg)
data_labels = np.concatenate((pos_labels, neg_labels))

binary_labels = np.zeros([len(data_labels), 2])

for i in range(len(data_labels)):
    if data_labels[i] == 1:
        binary_labels[i][1] = 1
    else:
        binary_labels[i][0] = 1


# Get mask length
mask_length = 0
for i in range(len(full_data)):
    if len(full_data[i]) > mask_length:
        mask_length = len(full_data[i])


In [0]:
# Pad data
data_padded = pad_sequences(full_data, maxlen=mask_length, dtype='object', padding='post', truncating='post', value=0)

# Shuffle data and get training and validation sets
indices = np.random.permutation(35267)
shuffled_data = data_padded[indices]
shuffled_labels = binary_labels[indices]


In [0]:
# Define hyper parameters
LSTM1_units = 32
LSTM2_units = 16
fully_connected_layer1_units = 32
fully_connected_layer2_units = 32
output_size = 2
learning_rate = 0.01



In [15]:
# Functional API model

# Input layer
inputs = Input(shape=(mask_length, 16), name='inputs')


# LSTM Layers
lstm1 = GRU(20, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(inputs)
lstm2 = GRU(10, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(lstm1)

#Flatten
flatten = Flatten()(lstm2)


# Fully connected layers
do1 = Dropout(0.1)(flatten)
fc1 = Dense(200, activation='sigmoid')(do1)
do2 = Dropout(0.1)(fc1)
fc2 = Dense(100, activation='sigmoid')(do2)

# Output layer
softmax = Dense(output_size, activation='softmax')(fc2)


# Compile model
model2 = Model(inputs=inputs, outputs=softmax)

model2.compile(optimizer='rmsprop',
               loss='binary_crossentropy',
               metrics=['accuracy'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 142, 16)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 142, 20)           2220      
_________________________________________________________________
gru_2 (GRU)                  (None, 142, 10)           930       
_________________________________________________________________
flatten_2 (Flatten)          (None, 1420)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1420)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               284200    
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
__________

In [0]:
def kfold(k, model, data, labels):
  
  epochs = 10
  length = len(data)
  accuracy = 0
  model.save_weights('initial_weights')
  
  for i in range(k):
    
    model.load_weights('initial_weights')
    
    #Get validation and training splits from data set
    lower_bound = int(i*(length/k))
    upper_bound = int((i+1)*(length/k))

    train_data = np.concatenate((data[0:lower_bound], data[upper_bound:length]))
    val_data = data[lower_bound:upper_bound]
    
    train_labels = np.concatenate((labels[0:lower_bound], labels[upper_bound:length]))
    val_labels = labels[lower_bound:upper_bound]
    
    history = model.fit(train_data, 
                      train_labels, 
                      epochs=epochs,
                      batch_size = 128,
                      validation_data=(val_data, val_labels))
    
    accuracy = accuracy + history.history['acc'][epochs-1]
    

  return accuracy / k

In [17]:
kfold_acc = kfold(5, model2, shuffled_data, shuffled_labels)
print('5-vold cross validation accuracy is: ', kfold_acc)

Train on 28214 samples, validate on 7053 samples
Epoch 1/10
28214/28214 [==============================] - 97s 3ms/step - loss: 0.4495 - acc: 0.7895 - val_loss: 0.3797 - val_acc: 0.8306
Epoch 2/10
28214/28214 [==============================] - 95s 3ms/step - loss: 0.3887 - acc: 0.8248 - val_loss: 0.3644 - val_acc: 0.8358
Epoch 3/10
28214/28214 [==============================] - 94s 3ms/step - loss: 0.3656 - acc: 0.8363 - val_loss: 0.3427 - val_acc: 0.8484
Epoch 4/10
28214/28214 [==============================] - 95s 3ms/step - loss: 0.3545 - acc: 0.8417 - val_loss: 0.3326 - val_acc: 0.8544
Epoch 5/10
28214/28214 [==============================] - 95s 3ms/step - loss: 0.3444 - acc: 0.8469 - val_loss: 0.3201 - val_acc: 0.8591
Epoch 6/10
28214/28214 [==============================] - 93s 3ms/step - loss: 0.3342 - acc: 0.8530 - val_loss: 0.3131 - val_acc: 0.8642
Epoch 7/10
28214/28214 [==============================] - 94s 3ms/step - loss: 0.3283 - acc: 0.8567 - val_loss: 0.3131 - val_acc:

In [0]:
def perfeval(predictions, Y_test, verbose=0):
 class_label = np.uint8(np.argmax(predictions,axis=1))
 R = np.asarray(np.uint8([sublist[1] for sublist in Y_test]))
 CM = metrics.confusion_matrix(R, class_label, labels=None)
 
 CM = np.double(CM)
 acc = (CM[0][0]+CM[1][1])/(CM[0][0]+CM[0][1]+CM[1][0]+CM[1][1])
 se = (CM[0][0])/(CM[0][0]+CM[0][1])
 sp = (CM[1][1])/(CM[1][0]+CM[1][1])
 f1 = (2*CM[0][0])/(2*CM[0][0]+CM[0][1]+CM[1][0])
 ppv = (CM[0][0])/(CM[0][0]+CM[1][0])
 mcc = (CM[0][0]*CM[1][1]-CM[0][1]*CM[1][0])/np.sqrt((CM[0][0]+CM[0][1])*(CM[0][0]+CM[1][0])*(CM[0][1]+CM[1][1])*(CM[1][0]+CM[1][1]))
 gmean = np.sqrt(se*sp)
 auroc = metrics.roc_auc_score(Y_test[:,0],predictions[:,0])
 aupr = metrics.average_precision_score(Y_test[:,0],predictions[:,0],average="micro")
  
 if verbose == 1:
  print("SE:","{:.3f}".format(se),"SP:","{:.3f}".format(sp),"F-Score:","{:.3f}".format(f1), "PPV:","{:.3f}".format(ppv),"gmean:","{:.3f}".format(gmean),"AUROC:","{:.3f}".format(auroc), "AUPR:","{:.3f}".format(aupr))
 
 return [se,sp,f1,ppv,gmean,auroc,aupr,CM]

In [19]:
from sklearn import metrics

# Get advanced metrics
preds = model2.predict(shuffled_data[0:5000])
met = perfeval(preds, shuffled_labels[0:5000], 1)

SE: 0.877 SP: 0.905 F-Score: 0.886 PPV: 0.894 gmean: 0.891 AUROC: 0.957 AUPR: 0.956
